<a href="https://colab.research.google.com/github/GabeAspir/Patent-Prior-Art-Finder/blob/main/4_SimilarityTechniques/Getting_into_metrics_Ephraim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text methodologies
* Importing new data
* Tokenizing



In [25]:
import pandas
from google.colab import files
from google.colab import drive
from sklearn.metrics.pairwise import pairwise_distances
import io
import re
import typing

In [3]:
# This requires re-authentitcating every time, so we'll stick to local storage
# drive.mount('/content/drive')
# %cd /content/drive/My Drive/bq-results-20210616-103551-n0c3m546u1li/

# pt= pandas.read_csv('bq-results-20210614-101306-uybbgfv0jf8q.csv')

In [4]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
pt = pandas.read_csv(io.BytesIO(uploaded['tenmatch.csv']))

Saving tenmatch.csv to tenmatch.csv
User uploaded file "tenmatch.csv" with length 492072 bytes


In [71]:
uploaded2 = files.upload()

for fn in uploaded2.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded2[fn])))
moarPat = pandas.read_csv(io.BytesIO(uploaded2['secondpattents.csv']))

Saving secondpattents.csv to secondpattents.csv
User uploaded file "secondpattents.csv" with length 4847362 bytes


In [72]:
moarPat

,Publication_Number,Abstract,Description,Claim
0,US-6573393-B1,The invention relates to a method for producin...,The invention relates to a process for the pre...,What is claimed is: \n \n 1. A proc...
1,US-7295414-B2,A power output device includes a bridged outpu...,CROSS-REFERENCE TO RELATED APPLICATION \n Th...,"1. A power output device, with a protection fu..."
2,US-6597387-B1,"A thermal printer includes a printer body, a t...",BACKGROUND OF INVENTION \n 1. Field of the...,What is claimed is: \n \n 1. A ther...
3,US-6786308-B1,An anti-lock brake system for a bicycle having...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. An ant...
4,US-6958450-B2,"Apparatus and systems, as well as fabrication ...",TECHNICAL FIELD \n The subject matter rela...,"1. An apparatus, comprising:\n a first compone..."
...,...,...,...,...
95,US-6569053-B2,A drive axle assembly is provided that include...,BACKGROUND OF THE INVENTION \n This invent...,What is claimed is: \n \n 1. A driv...
96,US-6853556-B1,"A motherboard assembly includes a motherboard,...",BACKGROUND \n A typical circuit board assemb...,1. An ejector assembly comprising:\n a housing...
97,US-7029800-B2,A static resistant reticle comprises a substra...,BACKGROUND OF THE INVENTION \n 1. Field of...,1. A static resistant reticle for use in integ...
98,US-7067547-B1,The present invention relates to contact lens ...,This application claims the benefit of Provisi...,1. A lens care solution comprising:\n an aqueo...


#Tokenization

In [6]:
  stop_words= {'','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}

I added '' to the stopwords to avoid the case where a short first word turns into an empty list item

In [7]:
def tokenize(input):
  out = input.lower()
  out= re.sub(r'\b\w{1,2}\b','',out) #remove anything not a word of length 2+
  out= re.sub(r"[0-9]+","_NUM_",out) #substitute _NUM_ for any block of consecutive number chars
  print(out)
  words= re.split('\W+', out) #Might need to change to pandas split at some point
                            # Note capital W is "Not word"= [a-zA-Z0-9_]
  words= list(filter(lambda s: s not in stop_words,words)) # why list not set?  ¯\_(ツ)_/¯
  #out = list(filter (lambda s: len (s) > 1, out))
  return words
print(tokenize("I had. 2 some text to ze z 25  100000 tokenize"))

 had.  some text      _NUM_ tokenize
['text', '_NUM_', 'tokenize']


In [8]:
atext= tokenize("I had. 2 some text to ze z 25  100000 tokenize")
adict= ['text','tokenize','pandas']

 had.  some text      _NUM_ tokenize


In [70]:
pt

,Publication_Number,Abstract,Description,Claim,tokenized,Bagged
0,US-9428454-B2,A process for the complete or partial oxygenat...,This application is a non-provisional applicat...,What is claimed is: \n \n 1. A proce...,"[process, complete, partial, oxygenation, hydr...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
1,US-9437520-B2,A semiconductor device includes a semiconducto...,TECHNICAL FIELD \n The technology disclose...,The invention claimed is: \n \n 1. A...,"[semiconductor, device, includes, semiconducto...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
2,US-9447236-B2,The present invention relates to a method of m...,This application is a National Stage entry of ...,What is claimed is: \n \n 1. A metho...,"[present, invention, relates, method, manufact...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
3,US-9449736-B2,A high dielectric contrast composition for par...,RELATED APPLICATIONS \n This application c...,The invention claimed is: \n \n 1. A...,"[high, dielectric, contrast, composition, part...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
4,US-2008199533-A1,The present invention relates to a cosmetic an...,RELATED APPLICATION \n This application is...,1 . A cosmetic and skin care formulation compr...,"[present, invention, relates, cosmetic, skin, ...","{'soursop': 1, 'scab': 0, 'projection': 0, 'mo..."
5,US-PP25006-P3,A new and distinct Malus domestica apple tre...,Latin name of the genus and species of the pla...,We claim: \n \n 1. A new and distinc...,"[new, distinct, malus, domestica, apple, tree,...","{'soursop': 0, 'scab': 1, 'projection': 0, 'mo..."
6,US-2001032345-A1,The present invention relates to a variety of ...,BACKGROUND OF THE INVENTION \n [0001] 1...,I claim: \n \n 1 . A variety of a...,"[present, invention, relates, variety, apple, ...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
7,US-2003195014-A1,A portable telephone set includes a telephone ...,BACKGROUND OF THE INVENTION \n [0001] 1...,What is claimed is: \n \n 1 . A p...,"[portable, telephone, set, includes, telephone...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
8,US-2003009852-A1,A hinge assembly ( 1 ) used for a foldable cel...,TECHNICAL FIELD \n [0001] This inventio...,1 . A hinge assembly comprising a pair of abut...,"[hinge, assembly, used, foldable, cellular, te...","{'soursop': 0, 'scab': 0, 'projection': 3, 'mo..."
9,US-2005155182-A1,A hinge device for coupling two units movable ...,FIELD OF THE INVENTION \n The present in...,1 . A hand-held electronic device comprising: ...,"[hinge, device, coupling, two, units, movable,...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."


In [12]:
def pTokenize(input): #Now bec pandas series are slightly different, and apparantly not itterating over the lists as strings is better (see: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas) 
  out = input.str.lower()
  out= out.str.replace(r"[0-9]+","_NUM_") #substitute _NUM_ for any block of consecutive number chars
  out= out.str.replace(r'\b\w{1,2}\b',"") #remove anything not a word of length 2+
  out= out.str.split('\W+') # Note capital W is "Not word"= [a-zA-Z0-9_]
  # Couldn't figure this out <<<<<<<<<<<
  #out[:]= list(filter(lambda s: s not in stop_words, out[:]))
  for r in out:
      #print(r)
      for s in r:
        if(s in stop_words):
          print(s,end="/")
  r= [s for s in r if not s in stop_words]
  print(out[0])
  print()
  #newlist= [s for s in out[] if not s in stop_words ]
  # filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  # print(out[0])
  # print(out.apply (lambda s: len (s) > 1 )[0])
  #out = out[out.apply (lambda s: len (s) > 1 )]
  return out
print(pTokenize(pt['Abstract']))

/for/the/and/the/and/under/the/the/and/both/and/the/out/from/the/the/the/while///having/and/which/the/the/that/the/the/the/the/the/the/the/the/are/not/the/the//the/more/the/and/the/having/and/and/and/being/for/after///for/that/and/into/the/and/the//the/and/from/and/from/and/and/from/and///and/and/and//the/and/the/the/and/the/the/the/the/the/the/the/the/the/for/the/for/the/the/the/with/the/are/with/each/from/about///and/the/with/and/the/and/has/the/the/the/with/which/with/the/for/and/the/for/the/the/out/from/the/and///for/are/the/where/the/the/and/the/the/are/with/each/other/the/into/with/the/both/the/when/the/are/the/and/the/the/and/both/the/should/and///for/and/with/the/such/the/about/and/about/the/that/the/the/and/that/the/the/the/for/the/against/the/and/for/the/against/the//['', 'process', 'for', 'the', 'complete', 'partial', 'oxygenation', 'hydrocarbons', 'comprises', 'contacting', '_NUM_', '_NUM_', 'hydrocarbon', 'molecular', 'oxygen', 'and', 'hydrogen', 'peroxide', 'the', 'presen

In [13]:
#pt['tokenized']=pTokenize(pt['claim_txt'])
pt['tokenized']=pt['Publication_Number']
for index, row in pt.iterrows():
  pt['tokenized'][index]= tokenize(pt['Abstract'][index])

 process for the complete  partial oxygenation  hydrocarbons comprises contacting    -   hydrocarbon, molecular oxygen, and hydrogen peroxide,  the presence  water and  heterogeneous catalyst, under conditions suitable  convert the   -   hydrocarbon   least one corresponding   -   oxygenate product, wherein the heterogeneous catalyst provides confinement and contains both brønsted-lowry acid centers and lewis acid centers. the reaction may  carried out   temperature ranging from ° .  ° . the use  molecular oxygen increases the economic attractiveness  the process while also improving yield.
 semiconductor device includes  semiconductor element having  rectangular shape   plan view, and  fixed member  which the semiconductor element  fixed. the semiconductor element  disposed  that  rectangular face  the semiconductor element  faced toward  surface  the fixed member.  part  the rectangular face  the semiconductor element  fixed  the surface  the fixed member.  least corner parts  the re

In [ ]:
pt

In [ ]:
acell= pt['tokenized'][0]
acell

In [15]:
def bow(series,corpus =None): # Takes a tokenized series
  if corpus is None:
    corpus = getCorpus(series)
  counts=[]
  for r in series:
    count={}
    for w in corpus:
      count[w]=r.count(w)
    counts.append(count)
  return counts
def getCorpus(series):
  corpus=set()
  for r in series:
    corpus.update(r)
  print("corpus: "+str(corpus) )
  return corpus
atext= tokenize("I had. 2 some text to ze z 25  100000 tokenize")
adict= ['text','tokenize','pandas']
bdict= ['unrealted','cellulardevice','talkingpoint']
ser= [adict,bdict]
print(bow(ser))

 had.  some text      _NUM_ tokenize
corpus: {'unrealted', 'pandas', 'talkingpoint', 'text', 'tokenize', 'cellulardevice'}
[{'unrealted': 0, 'pandas': 1, 'talkingpoint': 0, 'text': 1, 'tokenize': 1, 'cellulardevice': 0}, {'unrealted': 1, 'pandas': 0, 'talkingpoint': 1, 'text': 0, 'tokenize': 0, 'cellulardevice': 1}]


Adding binary BOW as new column

In [20]:
pt

,Publication_Number,Abstract,Description,Claim,tokenized,Bagged
0,US-9428454-B2,A process for the complete or partial oxygenat...,This application is a non-provisional applicat...,What is claimed is: \n \n 1. A proce...,"[process, complete, partial, oxygenation, hydr...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
1,US-9437520-B2,A semiconductor device includes a semiconducto...,TECHNICAL FIELD \n The technology disclose...,The invention claimed is: \n \n 1. A...,"[semiconductor, device, includes, semiconducto...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
2,US-9447236-B2,The present invention relates to a method of m...,This application is a National Stage entry of ...,What is claimed is: \n \n 1. A metho...,"[present, invention, relates, method, manufact...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
3,US-9449736-B2,A high dielectric contrast composition for par...,RELATED APPLICATIONS \n This application c...,The invention claimed is: \n \n 1. A...,"[high, dielectric, contrast, composition, part...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
4,US-2008199533-A1,The present invention relates to a cosmetic an...,RELATED APPLICATION \n This application is...,1 . A cosmetic and skin care formulation compr...,"[present, invention, relates, cosmetic, skin, ...","{'soursop': 1, 'scab': 0, 'projection': 0, 'mo..."
5,US-PP25006-P3,A new and distinct Malus domestica apple tre...,Latin name of the genus and species of the pla...,We claim: \n \n 1. A new and distinc...,"[new, distinct, malus, domestica, apple, tree,...","{'soursop': 0, 'scab': 1, 'projection': 0, 'mo..."
6,US-2001032345-A1,The present invention relates to a variety of ...,BACKGROUND OF THE INVENTION \n [0001] 1...,I claim: \n \n 1 . A variety of a...,"[present, invention, relates, variety, apple, ...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
7,US-2003195014-A1,A portable telephone set includes a telephone ...,BACKGROUND OF THE INVENTION \n [0001] 1...,What is claimed is: \n \n 1 . A p...,"[portable, telephone, set, includes, telephone...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
8,US-2003009852-A1,A hinge assembly ( 1 ) used for a foldable cel...,TECHNICAL FIELD \n [0001] This inventio...,1 . A hinge assembly comprising a pair of abut...,"[hinge, assembly, used, foldable, cellular, te...","{'soursop': 0, 'scab': 0, 'projection': 3, 'mo..."
9,US-2005155182-A1,A hinge device for coupling two units movable ...,FIELD OF THE INVENTION \n The present in...,1 . A hand-held electronic device comprising: ...,"[hinge, device, coupling, two, units, movable,...","{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."


In [17]:
#pt.reset_index(drop=True)
pt['Bagged']= bow(pt['tokenized'])
pt['Bagged']
# pt= pt.assign(bow(''))

corpus: {'soursop', 'scab', 'projection', 'molecular', 'white', 'multiple', 'coupling', 'sapote', 'butter', 'dioxide', 'confinement', 'wampi', 'oolong', 'direction', 'comprises', 'shape', 'aligned', 'neutral', 'passion', 'strong', 'complex', 'electric', 'imara', 'arcuate', 'corresponding', 'stage', 'communication', 'okari', 'processing', 'mixtures', 'pineapple', 'oxygenate', 'formulation', 'many', 'sapodilla', 'aroma', 'selection', 'drooping', 'coconut', 'velvet', 'including', 'corner', 'provides', 'field', 'characterized', 'freshwater', 'pink', 'peroxide', 'colored', 'respectively', 'suitable', 'yellowish', 'short', 'pearl', 'singe', 'urging', 'small', 'temperature', 'herbal', 'due', 'habit', 'taken', 'plantain', 'carbon', 'bias', 'thereby', 'oxygen', 'brønsted', 'fruit', 'tolerance', 'apple', 'consisting', 'ramified', 'rectangular', 'surface', 'peanut', 'carried', 'members', 'large', 'line', 'economic', 'frequently', 'dielectric', 'device', 'fuji', 'red', 'hinge', 'wherein', 'accommo

0    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
1    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
2    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
3    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
4    {'soursop': 1, 'scab': 0, 'projection': 0, 'mo...
5    {'soursop': 0, 'scab': 1, 'projection': 0, 'mo...
6    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
7    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
8    {'soursop': 0, 'scab': 0, 'projection': 3, 'mo...
9    {'soursop': 0, 'scab': 0, 'projection': 0, 'mo...
Name: Bagged, dtype: object

In [31]:
# Didn't figure out
#jaccard= 1- pairwise_distances(pt, metric = "jaccard")
#jaccard_table= pandas.DataFrame(jaccard,inex=pt.columns,columns=pt.columns)

AttributeError: ignored

It seems like for Jaccard it will be way simpler to use a binary BOW, so lets set that up.

In [37]:
def bbow(input,corpus =None): #Binary bag of words 
  if corpus is None:
    corpus = getCorpus(input)
  output=[]
  for r in input:
    out= [s in r for s in corpus]
    output.append(out)
  return output 
atext= pt['tokenized']
print(bbow(atext))

corpus: {'soursop', 'scab', 'projection', 'molecular', 'white', 'multiple', 'coupling', 'sapote', 'butter', 'dioxide', 'confinement', 'wampi', 'oolong', 'direction', 'comprises', 'shape', 'aligned', 'neutral', 'passion', 'strong', 'complex', 'electric', 'imara', 'arcuate', 'corresponding', 'stage', 'communication', 'okari', 'processing', 'mixtures', 'pineapple', 'oxygenate', 'formulation', 'many', 'sapodilla', 'aroma', 'selection', 'drooping', 'coconut', 'velvet', 'including', 'corner', 'provides', 'field', 'characterized', 'freshwater', 'pink', 'peroxide', 'colored', 'respectively', 'suitable', 'yellowish', 'short', 'pearl', 'singe', 'urging', 'small', 'temperature', 'herbal', 'due', 'habit', 'taken', 'plantain', 'carbon', 'bias', 'thereby', 'oxygen', 'brønsted', 'fruit', 'tolerance', 'apple', 'consisting', 'ramified', 'rectangular', 'surface', 'peanut', 'carried', 'members', 'large', 'line', 'economic', 'frequently', 'dielectric', 'device', 'fuji', 'red', 'hinge', 'wherein', 'accommo

In [47]:
#trying to make a table of column X column
matrix= pandas.DataFrame(pt['Publication_Number'])
matrix

,Publication_Number
0,US-9428454-B2
1,US-9437520-B2
2,US-9447236-B2
3,US-9449736-B2
4,US-2008199533-A1
5,US-PP25006-P3
6,US-2001032345-A1
7,US-2003195014-A1
8,US-2003009852-A1
9,US-2005155182-A1


In [62]:
def func_Jaccard(thingOne,thingTwo):
  #output=[one is two for one,two in zip(thingOne,thingTwo)]
  output=0
  for one,two in zip(thingOne,thingTwo):
    if one is two:
      output +=1
  output= output / len(thingOne)
  return output
textA= [True, True, False]
textB= [False, True, True]
print(func_Jaccard(textA,textB))
textC= [True,True,True,True]
textD=[False,False,False,False]
print(func_Jaccard(textD,textC))

0.3333333333333333
0.0


In [64]:
def bJac(input_series,index_series):
  fintable=pandas.DataFrame(index_series)
  for r,name in zip(input_series,index_series):
    print(name)
    mysim=[]
    for rr in input_series:
      mysim.append(func_Jaccard(r,rr))
    fintable[name]=mysim
  return fintable
simTable= bJac(bbow(pt['tokenized']),pt['Publication_Number'])
simTable

corpus: {'soursop', 'scab', 'projection', 'molecular', 'white', 'multiple', 'coupling', 'sapote', 'butter', 'dioxide', 'confinement', 'wampi', 'oolong', 'direction', 'comprises', 'shape', 'aligned', 'neutral', 'passion', 'strong', 'complex', 'electric', 'imara', 'arcuate', 'corresponding', 'stage', 'communication', 'okari', 'processing', 'mixtures', 'pineapple', 'oxygenate', 'formulation', 'many', 'sapodilla', 'aroma', 'selection', 'drooping', 'coconut', 'velvet', 'including', 'corner', 'provides', 'field', 'characterized', 'freshwater', 'pink', 'peroxide', 'colored', 'respectively', 'suitable', 'yellowish', 'short', 'pearl', 'singe', 'urging', 'small', 'temperature', 'herbal', 'due', 'habit', 'taken', 'plantain', 'carbon', 'bias', 'thereby', 'oxygen', 'brønsted', 'fruit', 'tolerance', 'apple', 'consisting', 'ramified', 'rectangular', 'surface', 'peanut', 'carried', 'members', 'large', 'line', 'economic', 'frequently', 'dielectric', 'device', 'fuji', 'red', 'hinge', 'wherein', 'accommo

,Publication_Number,US-9428454-B2,US-9437520-B2,US-9447236-B2,US-9449736-B2,US-2008199533-A1,US-PP25006-P3,US-2001032345-A1,US-2003195014-A1,US-2003009852-A1,US-2005155182-A1
0,US-9428454-B2,1.000000,0.814925,0.782090,0.805970,0.668657,0.788060,0.689552,0.758209,0.758209,0.782090
1,US-9437520-B2,0.814925,1.000000,0.847761,0.889552,0.734328,0.865672,0.773134,0.841791,0.841791,0.877612
2,US-9447236-B2,0.782090,0.847761,1.000000,0.856716,0.701493,0.820896,0.740299,0.791045,0.791045,0.820896
3,US-9449736-B2,0.805970,0.889552,0.856716,1.000000,0.725373,0.862687,0.758209,0.832836,0.826866,0.862687
4,US-2008199533-A1,0.668657,0.734328,0.701493,0.725373,1.000000,0.725373,0.644776,0.683582,0.671642,0.701493
5,US-PP25006-P3,0.788060,0.865672,0.820896,0.862687,0.725373,1.000000,0.758209,0.808955,0.808955,0.838806
6,US-2001032345-A1,0.689552,0.773134,0.740299,0.758209,0.644776,0.758209,1.000000,0.710448,0.716418,0.734328
7,US-2003195014-A1,0.758209,0.841791,0.791045,0.832836,0.683582,0.808955,0.710448,1.000000,0.785075,0.832836
8,US-2003009852-A1,0.758209,0.841791,0.791045,0.826866,0.671642,0.808955,0.716418,0.785075,1.000000,0.808955
9,US-2005155182-A1,0.782090,0.877612,0.820896,0.862687,0.701493,0.838806,0.734328,0.832836,0.808955,1.000000


Now Lets try that again with a random sample

In [ ]:
moarPat['tokenized']=moarPat['Publication_Number']
for index, row in moarPat.iterrows():
  moarPat['tokenized'][index]= tokenize(moarPat['Abstract'][index])

simTable2= bJac(bbow(moarPat['tokenized']),moarPat['Publication_Number'])


In [74]:
simTable2

,Publication_Number,US-6573393-B1,US-7295414-B2,US-6597387-B1,US-6786308-B1,US-6958450-B2,US-7127953-B1,US-7286605-B2,US-7191661-B2,US-6457881-B1,US-6521394-B1,US-6583608-B2,US-6893307-B1,US-7161610-B2,US-7350975-B2,US-6568338-B2,US-6772388-B2,US-7017467-B1,US-7120378-B2,US-6568160-B1,US-7064532-B1,US-7159163-B2,US-7265476-B1,US-6629746-B2,US-6629754-B1,US-6695562-B2,US-6798371-B1,US-6890357-B2,US-6958638-B2,US-7085120-B2,US-6497158-B1,US-6592452-B1,US-6595750-B2,US-6712687-B1,US-7099909-B2,US-7199849-B2,US-7208728-B2,US-6532558-B1,US-6548687-B1,US-6639777-B1,...,US-6957584-B2,US-7116125-B2,US-7149352-B2,US-7295988-B1,US-6610878-B1,US-7035007-B2,US-7241951-B2,US-7320879-B2,US-6940732-B2,US-6593001-B1,US-7068273-B2,US-6688317-B2,US-6511167-B1,US-7258057-B2,US-6790170-B2,US-7103217-B2,US-6641592-B1,US-7009785-B2,US-7081955-B2,US-6638249-B1,US-6651665-B1,US-7321061-B2,US-6594454-B2,US-6732376-B2,US-7164016-B2,US-7066529-B2,US-7215995-B2,US-7076771-B2,US-7106437-B2,US-7108655-B2,US-7217735-B1,US-7344832-B2,US-6888309-B2,US-7049754-B2,US-6483869-B1,US-6569053-B2,US-6853556-B1,US-7029800-B2,US-7067547-B1,US-7345188-B2
0,US-6573393-B1,1.000000,0.969697,0.974432,0.974905,0.979167,0.972064,0.980587,0.974905,0.977273,0.970644,0.973485,0.965436,0.978693,0.973485,0.973485,0.975379,0.975852,0.968277,0.975852,0.969223,0.973011,0.965436,0.977273,0.973485,0.979640,0.976326,0.973011,0.971591,0.973958,0.963542,0.962121,0.970644,0.966383,0.974905,0.970170,0.966856,0.965909,0.981534,0.977273,...,0.973958,0.975852,0.974432,0.958333,0.983902,0.978693,0.976326,0.976799,0.977746,0.975379,0.977746,0.973485,0.981534,0.968277,0.966856,0.971591,0.958333,0.985322,0.965436,0.969223,0.969697,0.987216,0.970644,0.973958,0.981061,0.973485,0.959754,0.979167,0.973485,0.972538,0.976799,0.977273,0.977746,0.964015,0.955966,0.968277,0.979167,0.977273,0.980587,0.975852
1,US-7295414-B2,0.969697,1.000000,0.964015,0.961648,0.965909,0.958807,0.967330,0.961648,0.965909,0.955492,0.964962,0.951231,0.964489,0.960227,0.962121,0.960227,0.964489,0.955966,0.962595,0.959754,0.958807,0.954072,0.964015,0.956439,0.967330,0.964015,0.959754,0.963068,0.963542,0.948390,0.948864,0.955492,0.953125,0.959754,0.957860,0.955492,0.953598,0.966383,0.967803,...,0.965436,0.964489,0.965909,0.942235,0.968750,0.964489,0.962121,0.961648,0.965436,0.961174,0.963542,0.960227,0.968277,0.951231,0.952652,0.959280,0.947917,0.973011,0.952178,0.955966,0.955492,0.972064,0.958333,0.959754,0.964962,0.959280,0.951231,0.964962,0.959280,0.960227,0.961648,0.963068,0.965436,0.951705,0.944602,0.955966,0.965909,0.963068,0.964489,0.959754
2,US-6597387-B1,0.974432,0.964015,1.000000,0.967330,0.969697,0.963542,0.972064,0.967330,0.971591,0.962121,0.966856,0.955966,0.970170,0.963068,0.964962,0.964015,0.965436,0.959754,0.966383,0.959754,0.964489,0.956913,0.968750,0.964015,0.972064,0.968750,0.963542,0.965909,0.964489,0.954072,0.952652,0.960227,0.959754,0.964489,0.962595,0.958333,0.956439,0.972064,0.969697,...,0.967330,0.964489,0.967803,0.946023,0.973485,0.970170,0.967803,0.965436,0.970170,0.966856,0.967330,0.964962,0.974905,0.958807,0.959280,0.964015,0.952652,0.976799,0.958807,0.960701,0.961174,0.976799,0.964015,0.966383,0.969697,0.964962,0.951231,0.969697,0.965909,0.964015,0.965436,0.966856,0.970170,0.954545,0.946496,0.960701,0.971591,0.969697,0.969223,0.964489
3,US-6786308-B1,0.974905,0.961648,0.967330,1.000000,0.971117,0.967803,0.972538,0.967803,0.970170,0.961648,0.965436,0.955492,0.970644,0.965436,0.965436,0.964489,0.965909,0.963068,0.966856,0.959280,0.964015,0.964015,0.969223,0.962595,0.971591,0.969223,0.964962,0.963542,0.964962,0.956439,0.956913,0.960701,0.961174,0.964962,0.960227,0.960701,0.956913,0.971591,0.969223,...,0.965909,0.965909,0.966383,0.947443,0.973958,0.970644,0.967330,0.965909,0.969697,0.965436,0.968750,0.964489,0.974432,0.958333,0.958807,0.963542,0.953125,0.979167,0.958333,0.961174,0.964489,0.977273,0.968277,0.964962,0.970170,0.967330,0.949811,0.971117,0.965436,0.965436,0.965909,0.969223,0.971591,0.957860,

In [80]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
pat = pandas.read_csv(io.BytesIO(uploaded['thirdpattents.csv']))


Saving thirdpattents.csv to thirdpattents (1).csv
User uploaded file "thirdpattents.csv" with length 10474323 bytes


In [82]:
pat

,Publication_Number,Abstract,Description,Claim,tokenized,Bagged
0,US-9428454-B2,A process for the complete or partial oxygenat...,This application is a non-provisional applicat...,What is claimed is: \n \n 1. A proce...,US-9428454-B2,"{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
1,US-9437520-B2,A semiconductor device includes a semiconducto...,TECHNICAL FIELD \n The technology disclose...,The invention claimed is: \n \n 1. A...,US-9437520-B2,"{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
2,US-9447236-B2,The present invention relates to a method of m...,This application is a National Stage entry of ...,What is claimed is: \n \n 1. A metho...,US-9447236-B2,"{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
3,US-9449736-B2,A high dielectric contrast composition for par...,RELATED APPLICATIONS \n This application c...,The invention claimed is: \n \n 1. A...,US-9449736-B2,"{'soursop': 0, 'scab': 0, 'projection': 0, 'mo..."
4,US-2008199533-A1,The present invention relates to a cosmetic an...,RELATED APPLICATION \n This application is...,1 . A cosmetic and skin care formulation compr...,US-2008199533-A1,"{'soursop': 1, 'scab': 0, 'projection': 0, 'mo..."
...,...,...,...,...,...,...
95,US-6569053-B2,A drive axle assembly is provided that include...,BACKGROUND OF THE INVENTION \n This invent...,What is claimed is: \n \n 1. A driv...,US-6569053-B2,NaN
96,US-6853556-B1,"A motherboard assembly includes a motherboard,...",BACKGROUND \n A typical circuit board assemb...,1. An ejector assembly comprising:\n a housing...,US-6853556-B1,NaN
97,US-7029800-B2,A static resistant reticle comprises a substra...,BACKGROUND OF THE INVENTION \n 1. Field of...,1. A static resistant reticle for use in integ...,US-7029800-B2,NaN
98,US-7067547-B1,The present invention relates to contact lens ...,This application claims the benefit of Provisi...,1. A lens care solution comprising:\n an aqueo...,US-7067547-B1,NaN


In [83]:
pat = pandas.read_csv(io.BytesIO(uploaded['thirdpattents.csv']))
pat['tokenized']=pat['Publication_Number']
for index, row in pat.iterrows():
  pat['tokenized'][index]= tokenize(pat['Abstract'][index])
pat=pandas.concat([pt,pat,moarPat])
simTable2= bJac(bbow(pat['tokenized']),pat['Publication_Number'])

 surface machining device for  light guide plate  disclosed, including:  support  ;   driving component    mounted  the support  ;   push unit    driven  the driving component  ,  configured  push the light guide plate      pre-set position;  positioning component    driven  the driving component  ,  configured  position the light guide plate     the pre-set position;  bearing component    mounted  the support  ,  configured  bear the light guide plate  ;  and  pressing component    driven  the driving component  ,  configured  press the light guide plate    onto the bearing component  .  with such device,  ultrathin light guide plate    can  machined  form  microstructure  the surface  the light guide plate  ,  with high productivity, and low cost.
 surface machining device for  light guide plate  disclosed, including:  support ;  driving component  mounted  the support ;  push unit  driven  the driving component , configured  push the light guide plate    pre-set position;  positioni

In [85]:
smtbl= simTable2.to_csv('smtbl.csv')
files.download('smtbl.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [86]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Day2-GoldSet.csv to Day2-GoldSet.csv
User uploaded file "Day2-GoldSet.csv" with length 471285 bytes


In [88]:
pat10 = pandas.read_csv(io.BytesIO(uploaded['Day2-GoldSet.csv']))
pat10['tokenized']=pat10['Publication_Number']
for index, row in pat10.iterrows():
  pat10['tokenized'][index]= tokenize(pat10['Abstract'][index])
simTable2= bJac(bbow(pat10['tokenized']),pat10['Publication_Number'])


 keyboard apparatus  including  plurality  keys and  plurality  mass members provided under the respective keys   pivotably supported   keyboard frame being  support member, each  the mass members extends along  longitudinal direction  the key and has  mass concentration part  its rear end portion, and when not driven   mass driving part  the key, each  the mass members    inclined state with the mass concentration part being  the lowest descended position. the keyboard frame has  rib provided between two adjacent mass members among the plural mass members, and  least part   lower edge  the rib  formed along lower edges  the mass members  the inclined state.
 easy-pull type swivel plug includes  body,  rotatable disk and  handle. the rotatable disk  rotatably provided  one side  the body. the rotatable disk  provided with two conductive pieces. the two conductive pieces extend beyond the body. the handle  pivoted  the other side  the body opposite  the rotatable disk   liftable and fol

In [89]:
simTable2

,Publication_Number,US-7365259-B2,US-7556524-B2,US-7338315-B2,US-6843642-B2,US-9433212-B2,US-5536436-A,US-2015111807-A1,US-7605322-B2,US-7205268-B2,US-6910186-B2
0,US-7365259-B2,1.000000,0.755627,0.774920,0.807074,0.842444,0.749196,0.790997,0.768489,0.778135,0.617363
1,US-7556524-B2,0.755627,1.000000,0.768489,0.800643,0.842444,0.749196,0.790997,0.762058,0.778135,0.617363
2,US-7338315-B2,0.774920,0.768489,1.000000,0.813505,0.848875,0.762058,0.797428,0.749196,0.784566,0.630225
3,US-6843642-B2,0.807074,0.800643,0.813505,1.000000,0.893891,0.813505,0.855305,0.813505,0.842444,0.675241
4,US-9433212-B2,0.842444,0.842444,0.848875,0.893891,1.000000,0.861736,0.877814,0.848875,0.877814,0.717042
5,US-5536436-A,0.749196,0.749196,0.762058,0.813505,0.861736,1.000000,0.836013,0.762058,0.836013,0.643087
6,US-2015111807-A1,0.790997,0.790997,0.797428,0.855305,0.877814,0.836013,1.000000,0.790997,0.864952,0.659164
7,US-7605322-B2,0.768489,0.762058,0.749196,0.813505,0.848875,0.762058,0.790997,1.000000,0.790997,0.630225
8,US-7205268-B2,0.778135,0.778135,0.784566,0.842444,0.877814,0.836013,0.864952,0.790997,1.000000,0.672026
9,US-6910186-B2,0.617363,0.617363,0.630225,0.675241,0.717042,0.643087,0.659164,0.630225,0.672026,1.000000
